# 수강 시간표를 구글 캘린더에 연동하기
* 수강 시간표를 구글 캘린더에 연동하기 위해서는 다음의 절차가 필요합니다.
    1. 시간표가 적용되는 학기 기간 정보, 수강 과목 정보를 입력합니다.
    2. 입력한 정보를 토대로 iCalendar(.ics) 파일을 생성합니다.
    3. 생성된 파일을 구글 캘린더에 업로드합니다.

------
### iCalendar 파일 생성 방법 1: 리스트에 담겨있는 수업 정보 이용하기

In [14]:
# import 해야하는 외부 라이브러리를 설치합니다.
!pip install icalendar pytz

Defaulting to user installation because normal site-packages is not writeable


In [15]:
"""
아래 코드는 리스트에 담겨있는 수업 정보를 사용하여
수강 시간표가 들어있는 iCalendar 파일을 생성합니다.
"""


from icalendar import Calendar, Event
from datetime import datetime, timedelta
import pytz

# 수업 정보
classes = [
    {"name": "교통정보공학및실습II", "day": "화요일", "start_time": "10:00", "end_time": "11:50", "location": "B14-618"},
    {"name": "실용영어회화중급", "day": "수요일", "start_time": "10:00", "end_time": "11:50", "location": "D15-503"},
    {"name": "농림수문원격탐사및실습", "day": "화요일", "start_time": "14:00", "end_time": "17:50", "location": "B14-618"},
    {"name": "수치사진측량학및실습", "day": "수요일", "start_time": "14:00", "end_time": "17:50", "location": "B14-618"},
    {"name": "교통공학특론1", "day": "목요일", "start_time": "13:00", "end_time": "14:50", "location": "B14-615"},
    {"name": "실무전산(I)", "day": "목요일", "start_time": "15:00", "end_time": "16:50", "location": "B15-101"},
    {"name": "해양원격탐사및실습", "day": "금요일", "start_time": "14:00", "end_time": "17:50", "location": "B14-614"},
]

# 요일을 숫자로 변환
day_map = {
    "월요일": 0,
    "화요일": 1,
    "수요일": 2,
    "목요일": 3,
    "금요일": 4,
    "토요일": 5,
    "일요일": 6
}

# 학기 시작일 설정
semester_start = datetime(2024, 3, 4)

# 시간대 설정
timezone = pytz.timezone("Asia/Seoul")

cal = Calendar()

for cls in classes:
    # 수업의 시작 요일 계산
    start_day = semester_start + timedelta(days=day_map[cls["day"]] - semester_start.weekday())

    # 시작 시간과 종료 시간 계산
    start_time = datetime.strptime(cls["start_time"], "%H:%M").time()
    end_time = datetime.strptime(cls["end_time"], "%H:%M").time()

    # 시작 일시와 종료 일시 설정
    start_datetime = datetime.combine(start_day.date(), start_time)
    start_datetime = timezone.localize(start_datetime)
    end_datetime = datetime.combine(start_day.date(), end_time)
    end_datetime = timezone.localize(end_datetime)

    # 이벤트 생성
    event = Event()
    event.add("summary", cls["name"])
    event.add("dtstart", start_datetime)
    event.add("dtend", end_datetime)
    event.add("location", cls["location"])
    event.add("rrule", {"freq": "weekly", "until": timezone.localize(datetime(2024, 6, 27))})  # 학기 종료일 설정

    cal.add_component(event)

# iCalendar 파일로 저장
with open("class_schedule.ics", "wb") as f:
    f.write(cal.to_ical())

print("iCalendar 파일이 'class_schedule.ics'로 저장되었습니다.")

iCalendar 파일이 'class_schedule.ics'로 저장되었습니다.


------
### iCalendar 파일 생성 방법 2: input()으로 수업 정보를 입력받아 사용하기

In [16]:
# import 해야하는 외부 라이브러리를 설치합니다.
!pip install icalendar pytz

Defaulting to user installation because normal site-packages is not writeable


In [18]:
"""
아래 코드는 사용자로부터 수강 과목에 대한 정보를 입력받아
수강 시간표가 들어있는 iCalendar 파일을 생성합니다.

사용자가 정보를 입력하는 과정에서 오타 등으로 잘못 입력했음을 인지했다면
즉시 다시 입력할 수 있는 기능이 구현되어있습니다.

사용자가 iCalendar 생성에 Error를 발생시킬 수 있는 올바르지 않은 정보를 입력했다면
즉시 프로그램을 종료시키는 기능이 구현되어있습니다.
"""


from icalendar import Calendar, Event
from datetime import datetime, timedelta
import pytz
import sys


# 사용자가 한글로 입력한 weekday를 숫자로 반환합니다.
def get_weekday_number(day):
    day_map = {
        "월요일": 0,
        "화요일": 1,
        "수요일": 2,
        "목요일": 3,
        "금요일": 4,
        "토요일": 5,
        "일요일": 6
    }
    return day_map.get(day)


# 사용자로부터 학기 시작 날짜와 종료 날짜를 입력받습니다.
# 입력받은 날짜를 통해 datetime 객체를 반환합니다.
# 입력받은 날짜의 서식이 올바르지 않다면 Error 메세지를 출력하고 종료합니다. 
def get_semester():
    start_date = input("시간표 입력을 시작합니다! 우선 이번 학기 시작 날짜를 입력해주세요. (예: 2024-03-04): ")
    end_date = input("이번 학기 종료 날짜를 입력해주세요. (예: 2024-06-27): ")
    try:
        semester_start = datetime.strptime(start_date, "%Y-%m-%d")
        semester_end = datetime.strptime(end_date, "%Y-%m-%d")
        return semester_start, semester_end
    except ValueError:
        print("올바르지 않은 서식으로 입력하셨습니다. 다시 시도해주세요.")
        print(f"\n[입력하신 값]\n학기 시작 날짜: {start_date}\n학기 종료 날짜: {end_date}")
        print("\n날짜는 하이픈(-)을 이용해 'YYYY-MM-DD'으로 나타내는 날짜 형식이어야 합니다. (예: 2024-03-04)")
        sys.exit(1)


# 사용자로부터 수업의 이름, 요일, 시간, 장소 등의 과목정보를 입력받습니다.
# 입력된 과목정보의 요일 서식과 시간 서식이 올바른지 확인합니다.
# 올바르지 않은 서식이 있다면 Error 메세지를 출력하고 종료합니다.
def input_class_info(class_number):
    while True:
        name = input(f"{class_number}번째 수업의 이름을 입력해주세요.: ")
        day = input(f"{class_number}번째 수업의 요일을 입력해주세요. (예: 월요일): ")
        start_time = input(f"{class_number}번째 수업의 시작 시간을 입력해주세요. (예: 14:00): ")
        end_time = input(f"{class_number}번째 수업의 종료 시간을 입력해주세요. (예: 17:50): ")
        location = input(f"{class_number}번째 수업의 장소를 입력해주세요. (예: B14-618): ")
        formatted_name = f"{name} ({day}, {start_time}~{end_time}, {location})"

        try:
            if day not in ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]:
                raise ValueError
            datetime.strptime(start_time, "%H:%M")
            datetime.strptime(end_time, "%H:%M")
            return {"name": formatted_name, "day": day, "start_time": start_time, "end_time": end_time, "location": location}
        except ValueError:
            print(f"올바르지 않은 서식으로 입력하셨습니다. 다시 시도해주세요.")
            print(f"\n[입력하신 값]\n수업 이름: {name}\n수업 요일: {day}\n시작 시간: {start_time}\n종료 시간: {end_time}\n수업 장소: {location}")
            print("\n[올바른 서식 안내]")
            print("수업 이름: 어떠한 문자열이든 입력 가능합니다.")
            print("수업 요일: 한국어로 요일을 입력하며, '월요일'부터 '일요일'까지의 문자열이어야 합니다.")
            print("시작 시간: 콜론(:)을 이용해 '시간:분'으로 나타내는 24시간 형식이어야 합니다. (예: 14:00)")
            print("종료 시간: 콜론(:)을 이용해 '시간:분'으로 나타내는 24시간 형식이어야 합니다. (예: 17:50)")
            print("수업 장소: 어떠한 문자열이든 입력 가능합니다.")
            sys.exit(1)


# 과목정보를 담아줄 비어있는 리스트를 생성합니다.
# 올바르게 입력된 과목정보를 리스트에 담습니다.
# 사용자가 과목정보를 수정하기 원하면 다시 입력받아 수정된 정보를 리스트에 덮어씁니다.
# 과목정보를 더 이상 수정하고 싶지 않으면 반복문을 탈출합니다.
# 과목을 더 이상 추가하고 싶지 않으면 반복문을 탈출합니다.
# 수강 과목의 정보들을 담은 최종 리스트를 반환합니다.
def get_class_info():
    classes = []
    class_number = 1
    
    while True:
        class_info = input_class_info(class_number)
        classes.append(class_info)

        while True:
            modify = input(f"[{class_number}과목] {class_info['name']}을(를) 수정하시겠습니까? (예/아니오): ")
            if modify.lower() == "예":
                class_info = input_class_info(class_number)
                classes[class_number - 1] = class_info
            else:
                break

        more_classes = input(f"[{class_number}과목] {class_info['name']}이 성공적으로 입력되었습니다. 더 추가할 과목이 있습니까? (예/아니오): ")
        if more_classes.lower() != "예":
            break

        class_number += 1

    return classes


# Calendar()를 생성하고 한국 표준시를 사용합니다. 
# Event()를 만들기 위해 필요한 값(일정 이름, 날짜, 시간, 장소 등)들을 추출합니다.
# 과목 이름이 formatted_name으로 저장되어 있으므로, 순수한 과목명만 추출해 일정 이름으로 사용합니다.
# 각각의 Event()를 구성하는 요소를 add하고, 완성된 Calendar()를 반환합니다.
def create_calendar(classes, semester_start, semester_end):
    cal = Calendar()
    timezone = pytz.timezone("Asia/Seoul")
    for cls in classes:
        event = Event()
        simple_name = cls["name"].split(" (")[0]
        start_day = semester_start + timedelta(days=get_weekday_number(cls["day"]) - semester_start.weekday())
        start_time = datetime.strptime(cls["start_time"], "%H:%M").time()
        end_time = datetime.strptime(cls["end_time"], "%H:%M").time()
        start_datetime = datetime.combine(start_day.date(), start_time)
        start_datetime = timezone.localize(start_datetime)
        end_datetime = datetime.combine(start_day.date(), end_time)
        end_datetime = timezone.localize(end_datetime)

        event.add("summary", simple_name)
        event.add("dtstart", start_datetime)
        event.add("dtend", end_datetime)
        event.add("location", cls["location"])
        event.add("rrule", {"freq": "weekly", "until": timezone.localize(semester_end)})

        cal.add_component(event)

    return cal

# 학기 시작일, 종료일, 과목 정보를 입력받아 변수에 저장합니다.
# datetime 객체의 year, month 속성을 통해 연도,월 정보를 추출합니다.
# 추출한 월이 1~6월이면 1학기, 7~12월이면 2학기로 판단합니다.
# 입력받은 과목정보를 print 합니다.
# 연도와 학기 정보를 이용해 파일 이름을 생성합니다.
# 학기 정보와 과목 정보로 iCalendar 파일(.ics)을 작성합니다.
def main():
    semester_start, semester_end = get_semester()
    classes = get_class_info()
    year = semester_start.year
    month = semester_start.month
    semester = 1 if month <= 6 else 2

    print(f"\n{year}년 {semester}학기 시간표 입력이 완료되었습니다.\n")
    print("[입력한 과목 정보]")
    for i, cls in enumerate(classes, 1):
        print(f"{i}과목 : {cls['name']}")

    file_name = f"{year}_{semester}_Class_schedule.ics"

    cal = create_calendar(classes, semester_start, semester_end)
    with open(file_name, "wb") as f:
        f.write(cal.to_ical())
    print(f"\niCalendar 파일이 '{file_name}'로 저장되었습니다.")


# 현재 스크립트가 직접 실행되었을 때에만 main()을 호출하여 코드가 실행됩니다.
# 이 스크립트가 다른 스크립트에서 import 되었을 때는 main()이 호출되지 않습니다.
if __name__ == "__main__":
    main()


2024년 1학기 시간표 입력이 완료되었습니다.

[입력한 과목 정보]
1과목 : 교통정보공학및실습II (화요일, 10:00~11:50, B14-618)
2과목 : 농림수문원격탐사및실습 (화요일, 14:00~17:50, B14-618)
3과목 : 실용영어회화중급 (수요일, 10:00~11:50, D15-503)
4과목 : 수치사진측량학및실습 (수요일, 14:00~17:50, B14-618)
5과목 : 교통공학특론I (목요일, 13:00~14:50, B14-615)
6과목 : 실무전산(I) (목요일, 15:00~16:50, B15-101)
7과목 : 해양원격탐사및실습 (금요일, 14:00~17:50, B14-614)

iCalendar 파일이 '2024_1_Class_schedule.ics'로 저장되었습니다.


------
### 생성된 iCalendar 파일을 구글 캘린더에 연동하는 방법은 크게 2가지가 있습니다.

1. iCalendar 파일을 Google Calendar에 직접 업로드
    1. Google Calendar에 로그인합니다.
    2. 메인 화면의 왼쪽 패널에서 '다른 캘린더'를 찾아 옆의 '+' 버튼을 클릭합니다.
    3. '가져오기'를 선택합니다.
    4. '컴퓨터에서 파일 선택' 버튼을 클릭하여 생성된 '.ics' 파일을 업로드합니다.
    5. '가져오기' 버튼을 클릭합니다.

2. iCalendar 파일을 웹 서버에 업로드하고 URL로 Google Calendar에 연동
    1. iCalendar 파일을 Github와 같은 웹 서버에 업로드합니다.
    2. 업로드 한 파일의 Raw URL을 복사합니다.
    3. Google Calendar에 로그인합니다.
    4. 메인 화면의 왼쪽 패널에서 '다른 캘린더'를 찾아 옆의 '+' 버튼을 클릭합니다.
    5. 'URL로 추가'를 선택합니다.
    6. URL 입력란에 복사한 Raw URL을 붙여넣고 '캘린더 추가' 버튼을 클릭합니다.

#### 1번 방법은 일정이 '내 캘린더'에 추가됩니다.
#### 2번 방법은 일정이 '다른 캘린더'에 추가됩니다.